In [ ]:
!pip install pandas google-cloud-bigquery
!pip install pandas-gbq
import pandas as pd
from google.oauth2 import service_account
import pandas as pd
import pandas_gbq as gbq
credentials_path = '../token.json'

# Authenticate with your credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=['https://www.googleapis.com/auth/bigquery'])

# Set the credentials for pandas_gbq
gbq.context.credentials = credentials

In [14]:
from pandas_gbq import read_gbq
merged_data = read_gbq(f"""
SELECT *
FROM `capstone-398012.capstone.X_Y_merged_2`

""",project_id='capstone-398012', dialect='standard')

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [41]:
data = ['CompanyNumber', 'yyyy', 'mm', 'ni2talevel','ni2tatrend', 'sizelevel', 'sizetrend', 'm2b']
# temp = merged_data.copy()
merged_data= temp.copy()

In [50]:
merged_data.sort_values(by=['CompanyNumber', 'yyyy', 'mm'], inplace=True)


def rolling_average_with_min_count(series):
    rolling_window = series.rolling(window=48, min_periods=0)
    return rolling_window.mean()


level_variables = ['ni2talevel', 'sizelevel']

# Loop through each 'level' variable and apply the imputation logic
for variable in level_variables:
    imputed_column = merged_data.groupby('CompanyNumber')[variable].apply(rolling_average_with_min_count)
    
    merged_data[variable + '_imputed'] = imputed_column.reset_index(level=0, drop=True)
merged_data['sizelevel'].fillna(merged_data['sizelevel_imputed'], inplace=True)
merged_data['ni2talevel'].fillna(merged_data['ni2talevel_imputed'], inplace=True)


In [27]:
temp.columns

Index(['CompanyNumber', 'yyyy', 'mm', 'StkIndx', 'STInt', 'dtdlevel',
       'dtdtrend', 'liqnonfinlevel', 'liqnonfintrend', 'ni2talevel',
       'ni2tatrend', 'sizelevel', 'sizetrend', 'm2b', 'sigma', 'liqfinlevel',
       'lqfintrend', 'DTDmedianFin', 'DTDmedianNonFin', 'dummy297fin',
       'EventDate', 'EventType'],
      dtype='object')

In [51]:
merged_data.isna().sum()

CompanyNumber              0
yyyy                       0
mm                         0
StkIndx                47043
STInt                  17318
dtdlevel              415827
dtdtrend              415827
liqnonfinlevel        313582
liqnonfintrend        313582
ni2talevel            228614
ni2tatrend            295878
sizelevel             142513
sizetrend             160735
m2b                   304381
sigma                 289846
liqfinlevel            64824
lqfintrend             64824
DTDmedianFin           29919
DTDmedianNonFin       126689
dummy297fin                0
EventDate                  0
EventType                  0
ni2talevel_imputed    228614
sizelevel_imputed     142513
dtype: int64